Let’s work on the following:
1. Access the CB API (with my login info), which is inside Dropbox. Download the list of companies founded in 2016 and with category ‘Artificial Intelligence’. That should be 500sh companies, we start with a small sample.
2. For each of those companies, download all the NEWS that Crunchbase has. Try to put those news into an easy format: maybe a TXT file per news, and all TXT files of a company into one folder. And remove all non-relevant components (like Ads or other text). This will make it easier if somebody in the team will have to do some reading.
3. Once this looks good, we’ll do some NLP on the text. But let’s get the "raw data" first.
4. Please upload all material on Dropbox.

In [1]:
import json
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

import nltk
from nltk import *
from nltk.corpus import PlaintextCorpusReader
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import sys
import codecs
import string
import random
import re

import requests
import urllib
import bs4
from bs4 import BeautifulSoup
from readability.readability import Document # https://github.com/buriy/python-readability. Tried Goose, Newspaper (python libraries on Github). Bad results.
from http.cookiejar import CookieJar # 

import pycrunchbase
cb = pycrunchbase.CrunchBase('662e263576fe3e4ea5991edfbcfb9883')

/Users/ksun/Downloads/venv3/lib/python3.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [4]:
# df = pd.DataFrame(columns=["permalink"])

# url = 'https://api.crunchbase.com/v3.1/organizations/?user_key=662e263576fe3e4ea5991edfbcfb9883'
# # location given here 
# location = "philadephia"

# # defining a params dict for the parameters to be sent to the API 
# PARAMS = {'address':location} 

# while True:
#     try:
#         response = requests.get(url, PARAMS)
#         json = response.json()
#         for c in json['data']['items']:
#             permalink=c['properties']['permalink']
#             if(permalink!=None):
#                 categories_json = requests.get("https://api.crunchbase.com/v3.1/organizations/" 
#                                 + permalink+"/categories?user_key=662e263576fe3e4ea5991edfbcfb9883", PARAMS).json()
#                 for cat in categories_json['data']['items']:
#                     if(cat["properties"]["category_groups"]!=None):
#                         if("Artificial Intelligence" in cat["properties"]["category_groups"]):
#                             print(permalink)
#                             df = df.append({'permalink' : permalink} , ignore_index=True)
#         url = json['data']['paging']['next_page_url'] + "&user_key=662e263576fe3e4ea5991edfbcfb9883"
#         print(url)
#     except:
#         print("error: " + url)
        
# df

df = pd.read_csv('csv_export/organizations.csv')
print(df.columns)
df

Index(['company_name', 'roles', 'permalink', 'domain', 'homepage_url',
       'country_code', 'state_code', 'region', 'city', 'address', 'status',
       'short_description', 'category_list', 'category_group_list',
       'funding_rounds', 'funding_total_usd', 'founded_on', 'last_funding_on',
       'closed_on', 'employee_count', 'email', 'phone', 'facebook_url',
       'linkedin_url', 'cb_url', 'logo_url', 'twitter_url', 'aliases', 'uuid',
       'created_at', 'updated_at', 'primary_role', 'type'],
      dtype='object')


,company_name,roles,permalink,domain,homepage_url,country_code,state_code,region,city,address,...,linkedin_url,cb_url,logo_url,twitter_url,aliases,uuid,created_at,updated_at,primary_role,type
0,Grown Ocean,company,/organization/grown-ocean,grownocean.com,http://www.grownocean.com,USA,NY,New York City,New York,"79 Madison Avenue, 2nd Floor",...,https://www.linkedin.com/company/6440106?trk=t...,https://www.crunchbase.com/organization/grown-...,https://crunchbase-production-res.cloudinary.c...,https://twitter.com/GrownOceanApp,NaN,eea6f58a-b728-d416-7e38-6ca5bad23e2d,2016-04-26 20:25:23,2018-10-26 22:27:32,company,organization
1,Intel,"company,investor",/organization/intel,intel.com,http://www.intel.com/,USA,CA,SF Bay Area,Santa Clara,2200 Mission College Blvd,...,https://www.linkedin.com/company/intel-corpora...,https://www.crunchbase.com/organization/intel,https://crunchbase-production-res.cloudinary.c...,http://twitter.com/intel,N M Electronics,1e4f199c-363b-451b-a164-f94571075ee5,2008-02-06 06:28:14,2018-12-11 10:52:30,company,organization
2,Intercomp,company,/organization/intercomp,intercompcompany.com,http://www.intercompcompany.com/,USA,OH,Cleveland,Medina,3839 County Road 116,...,https://www.linkedin.com/company/906845,https://www.crunchbase.com/organization/intercomp,https://crunchbase-production-res.cloudinary.c...,NaN,NaN,6681b1b0-0cea-6a4a-820d-60b15793fa66,2015-07-26 19:09:56,2018-04-17 12:08:01,company,organization
3,Gruppo Editoriale L'Espresso Spa,company,/organization/gruppo-editoriale-lespresso-spa,gruppoespresso.it,http://www.gruppoespresso.it/en/nc.html,ITA,NaN,Rome,Rome,"Gruppo Editoriale L’Espresso Spa, Via Cristofo...",...,https://www.linkedin.com/company/952199,https://www.crunchbase.com/organization/gruppo...,https://crunchbase-production-res.cloudinary.c...,NaN,NaN,e74463f7-94a8-ab7b-ba31-f4cd57a16570,2012-04-24 19:27:30,2018-08-07 12:49:54,company,organization
4,LSI Logic,"company,investor",/organization/lsi,lsi.com,http://www.lsi.com,USA,CA,SF Bay Area,Milpitas,1621 Barber Lane,...,http://www.linkedin.com/company/lsi-corporation,https://www.crunchbase.com/organization/lsi,https://crunchbase-production-res.cloudinary.c...,http://twitter.com/LSICorporation,LSI Logic,aa320353-a1eb-5368-8a51-f39e27954f90,2008-05-23 04:19:43,2018-10-26 22:39:57,company,organization
5,Microsoft,"company,investor",/organization/microsoft,microsoft.com,http://www.microsoft.com,USA,WA,Seattle,Redmond,1 Microsoft Way,...,http://www.linkedin.com/company/microsoft,https://www.crunchbase.com/organization/microsoft,https://crunchbase-production-res.cloudinary.c...,http://twitter.com/Microsoft,msn,fd80725f-53fc-7009-9878-aeecf1e9ffbb,2014-05-12 12:00:58,2019-02-01 16:07:42,company,organization
6,3Com,"company,investor",/organization/3com,3com.com,http://www.3com.com,USA,MA,Boston,Marlborough,350 Campus Dr,...,http://in.linkedin.com/company/3com,https://www.crunchbase.com/organization/3com,https://crunchbase-production-res.cloudinary.c...,https://www.twitter.com/hpe,NaN,5e645e6c-d435-fd14-f82f-efa7a300cc86,2008-09-08 11:21:20,2018-08-21 09:12:34,company,organization
7,Compaq,"company,investor",/organization/compaq,compaq.com,http://www.compaq.com,USA,CA,SF Bay Area,Palo Alto,3000 Hanover Street,...,https://www.linkedin.com/company/hp,https://www.crunchbase.com/organization/compaq,https://crunchbase-production-res.cloudinary.c...,https://www.twitter.com/hpnews,NaN,10a3b2fd-b142-046b-7d8f-3b1aa4877aca,2008-08-26 04:03:18,2018-04-17 09:10:55,company,organization
8,BDS,company,/organization/bds-incorporated,NaN,NaN,USA,NC,Asheville,Lenoir,1260 Hidden Cove Lane,...,NaN,https://www.crunchbase.com/organization/bds-in...,https://crunchbase-production-res.cloudinary.c...,NaN,Business Data Solutions,1d410d6c-13ce-3d53-4e3f-be70dd5ffd54,2015-08-20 09:26:00,2019-01-13 04:32:16,company,organization
9,Ikro,company,/organization/ikro,ikro.com.br,https://www.ikro.com.br/,BRA,NaN,BRA - Other,Canoas,"Rua Lauro Muller, 110, São José Rio Grande do Sul",.

In [32]:
ai_df = pd.DataFrame(columns = df.columns)
for i in range(len(df)):
    try:
        if("Artificial Intelligence" in df['category_group_list'][i] and '2016' in df['founded_on'][i]):
            ai_df = ai_df.append(df.loc[i])
    except:
        pass
ai_df

,company_name,roles,permalink,domain,homepage_url,country_code,state_code,region,city,address,...,linkedin_url,cb_url,logo_url,twitter_url,aliases,uuid,created_at,updated_at,primary_role,type
19953,Ubiquity.ai,company,/organization/ubiquity-ai,ubiquity.ai,http://ubiquity.ai,NaN,NaN,NaN,NaN,NaN,...,https://www.linkedin.com/company/ubiquity.ai/,https://www.crunchbase.com/organization/ubiqui...,https://crunchbase-production-res.cloudinary.c...,NaN,NaN,aaa64a6f-2e07-4af1-b81e-dfbd362bd82f,2018-06-14 11:25:44,2018-10-27 00:21:13,company,organization
24707,Viralheat,"company,investor",/organization/viralheat,viralheat.com,http://www.viralheat.com,USA,CA,SF Bay Area,San Mateo,1510 Fashion Island Blvd. Ste. 260,...,https://www.linkedin.com/company/viralheat/,https://www.crunchbase.com/organization/viralheat,https://crunchbase-production-res.cloudinary.c...,http://twitter.com/viralheat,NaN,500ae0bc-dcb0-9d61-f228-92289c56a8ef,2009-05-25 21:49:10,2018-12-11 11:26:45,company,organization
49316,DealStart,company,/organization/dealstart,dealstart.com,https://dealstart.com,USA,NY,New York City,New York,"79 Madison Avenue, 7th Floor",...,https://www.linkedin.com/company/dealstart,https://www.crunchbase.com/organization/dealstart,https://crunchbase-production-res.cloudinary.c...,https://twitter.com/dealstart,NaN,a7fe8d37-4ab6-71e9-4a7b-3d0bc4569eeb,2014-03-11 00:39:35,2019-01-19 03:56:17,company,organization
53877,UniQreate,company,/organization/uniqreate-limited,uniqreate.com,http://www.uniqreate.com,USA,NY,New York City,New York,"353 Lexington Avenue, Suite 1605",...,https://www.linkedin.com/company/uniqreate,https://www.crunchbase.com/organization/uniqre...,https://crunchbase-production-res.cloudinary.c...,https://twitter.com/Uni_Qreate,"UQ,ReQon,Xtract",b85e315c-a61c-adc1-6a78-5c57300a4361,2015-09-22 07:43:15,2019-01-28 04:31:27,company,organization
57355,Really,company,/organization/really-really-inc,really.ai,https://really.ai/,USA,CA,SF Bay Area,Palo Alto,950 Page Mill Road,...,https://www.linkedin.com/company-beta/17978433/,https://www.crunchbase.com/organization/really...,https://crunchbase-production-res.cloudinary.c...,https://twitter.com/reallyreallyio,NaN,0441a298-919d-8999-2977-88a7b144ff06,2017-03-08 04:33:08,2018-10-26 23:19:07,company,organization
59758,Flyreel,company,/organization/flyreel,flyreel.com,http://flyreel.com,USA,CO,Denver,Denver,3000 Lawrence Street,...,https://www.linkedin.com/company/2933039,https://www.crunchbase.com/organization/flyreel,https://crunchbase-production-res.cloudinary.c...,NaN,NaN,9dad2e54-1b28-ddbb-9d5f-e4b1263deccc,2015-08-25 08:24:47,2018-10-26 22:17:06,company,organization
68330,Polte Corporation,company,/organization/polte,polte.com,https://www.polte.com/,USA,TX,Dallas,Richardson,801 E. Campbell Road\nSuite 580\n,...,https://www.linkedin.com/company/polte/,https://www.crunchbase.com/organization/polte,https://crunchbase-production-res.cloudinary.c...,https://twitter.com/PoLTECorp,Invisitrack,308560bf-494f-4ab4-8ce3-b7164450fa62,2018-04-09 04:04:04,2018-10-26 23:07:46,company,organization
70151,ParallelM,company,/organization/parallel-machines,parallelm.com,http://www.parallelm.com,USA,CA,SF Bay Area,Sunnyvale,"1290 Oakmead Parkway, Suite #301",...,https://www.linkedin.com/company/parallel-mach...,https://www.crunchbase.com/organization/parall...,https://crunchbase-production-res.cloudinary.c...,https://twitter.com/parallelm_ai,ParallelM - The MLOps,caf033b7-c50a-d024-21af-c37e62b49d2e,2015-11-04 18:19:05,2019-01-10 02:38:14,company,organization
71373,Puttmee by Kausoft GmbH,company,/organization/kausoft-gmbh,puttmee.com,http://puttmee.com,DEU,NaN,Cologne,Cologne,Wiedstr. 8,...,https://www.linkedin.com/company/puttmee/,https://www.crunchbase.com/organization/kausof...,https://crunchbase-production-res.cloudinary.c...,http://twitter.com/puttmee,Puttmee,011e8ea0-ef22-4ed5-81ce-9aa2c8dc09a1,2017-11-16 16:54:54,2019-01-17 04:59:31,company,organization
72298,Browsi,company,/organization/browsi,gob

In [38]:
ai_df= ai_df.reset_index(drop=True)
ai_df

,company_name,roles,permalink,domain,homepage_url,country_code,state_code,region,city,address,...,linkedin_url,cb_url,logo_url,twitter_url,aliases,uuid,created_at,updated_at,primary_role,type
0,Ubiquity.ai,company,/organization/ubiquity-ai,ubiquity.ai,http://ubiquity.ai,NaN,NaN,NaN,NaN,NaN,...,https://www.linkedin.com/company/ubiquity.ai/,https://www.crunchbase.com/organization/ubiqui...,https://crunchbase-production-res.cloudinary.c...,NaN,NaN,aaa64a6f-2e07-4af1-b81e-dfbd362bd82f,2018-06-14 11:25:44,2018-10-27 00:21:13,company,organization
1,Viralheat,"company,investor",/organization/viralheat,viralheat.com,http://www.viralheat.com,USA,CA,SF Bay Area,San Mateo,1510 Fashion Island Blvd. Ste. 260,...,https://www.linkedin.com/company/viralheat/,https://www.crunchbase.com/organization/viralheat,https://crunchbase-production-res.cloudinary.c...,http://twitter.com/viralheat,NaN,500ae0bc-dcb0-9d61-f228-92289c56a8ef,2009-05-25 21:49:10,2018-12-11 11:26:45,company,organization
2,DealStart,company,/organization/dealstart,dealstart.com,https://dealstart.com,USA,NY,New York City,New York,"79 Madison Avenue, 7th Floor",...,https://www.linkedin.com/company/dealstart,https://www.crunchbase.com/organization/dealstart,https://crunchbase-production-res.cloudinary.c...,https://twitter.com/dealstart,NaN,a7fe8d37-4ab6-71e9-4a7b-3d0bc4569eeb,2014-03-11 00:39:35,2019-01-19 03:56:17,company,organization
3,UniQreate,company,/organization/uniqreate-limited,uniqreate.com,http://www.uniqreate.com,USA,NY,New York City,New York,"353 Lexington Avenue, Suite 1605",...,https://www.linkedin.com/company/uniqreate,https://www.crunchbase.com/organization/uniqre...,https://crunchbase-production-res.cloudinary.c...,https://twitter.com/Uni_Qreate,"UQ,ReQon,Xtract",b85e315c-a61c-adc1-6a78-5c57300a4361,2015-09-22 07:43:15,2019-01-28 04:31:27,company,organization
4,Really,company,/organization/really-really-inc,really.ai,https://really.ai/,USA,CA,SF Bay Area,Palo Alto,950 Page Mill Road,...,https://www.linkedin.com/company-beta/17978433/,https://www.crunchbase.com/organization/really...,https://crunchbase-production-res.cloudinary.c...,https://twitter.com/reallyreallyio,NaN,0441a298-919d-8999-2977-88a7b144ff06,2017-03-08 04:33:08,2018-10-26 23:19:07,company,organization
5,Flyreel,company,/organization/flyreel,flyreel.com,http://flyreel.com,USA,CO,Denver,Denver,3000 Lawrence Street,...,https://www.linkedin.com/company/2933039,https://www.crunchbase.com/organization/flyreel,https://crunchbase-production-res.cloudinary.c...,NaN,NaN,9dad2e54-1b28-ddbb-9d5f-e4b1263deccc,2015-08-25 08:24:47,2018-10-26 22:17:06,company,organization
6,Polte Corporation,company,/organization/polte,polte.com,https://www.polte.com/,USA,TX,Dallas,Richardson,801 E. Campbell Road\nSuite 580\n,...,https://www.linkedin.com/company/polte/,https://www.crunchbase.com/organization/polte,https://crunchbase-production-res.cloudinary.c...,https://twitter.com/PoLTECorp,Invisitrack,308560bf-494f-4ab4-8ce3-b7164450fa62,2018-04-09 04:04:04,2018-10-26 23:07:46,company,organization
7,ParallelM,company,/organization/parallel-machines,parallelm.com,http://www.parallelm.com,USA,CA,SF Bay Area,Sunnyvale,"1290 Oakmead Parkway, Suite #301",...,https://www.linkedin.com/company/parallel-mach...,https://www.crunchbase.com/organization/parall...,https://crunchbase-production-res.cloudinary.c...,https://twitter.com/parallelm_ai,ParallelM - The MLOps,caf033b7-c50a-d024-21af-c37e62b49d2e,2015-11-04 18:19:05,2019-01-10 02:38:14,company,organization
8,Puttmee by Kausoft GmbH,company,/organization/kausoft-gmbh,puttmee.com,http://puttmee.com,DEU,NaN,Cologne,Cologne,Wiedstr. 8,...,https://www.linkedin.com/company/puttmee/,https://www.crunchbase.com/organization/kausof...,https://crunchbase-production-res.cloudinary.c...,http://twitter.com/puttmee,Puttmee,011e8ea0-ef22-4ed5-81ce-9aa2c8dc09a1,2017-11-16 16:54:54,2019-01-17 04:59:31,company,organization
9,Browsi,company,/organization/browsi,gobrowsi.com,http://gobrowsi.com,ISR,NaN,Na

In [56]:
# location given here 
location = "philadephia"

# defining a params dict for the parameters to be sent to the API 
PARAMS = {'address':location} 
for i in range(len(ai_df)):
    permalink = ai_df['permalink'][i].split("/")[2]
    newpath = '/Users/ksun/downloads/patent-decision/ai-organizations/' + permalink
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    
        url = 'https://api.crunchbase.com/v3.1/organizations/' + permalink+'?relationships=news&user_key=662e263576fe3e4ea5991edfbcfb9883'
        response_json = requests.get(url, PARAMS).json()

        for n in response_json['data']['relationships']['news']['items']:
            try:
                uri=n['properties']['url']
                opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor)
                opener.addheaders = [('User-agent', 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/535.7 (KHTML, like Gecko) Chrome/16.0.912.77 Safari/535.7')]
                html =  opener.open(uri).read().decode('utf-8')
                readable_article = Document(html).summary()
                soup = BeautifulSoup(readable_article, "lxml")
                text = soup.get_text()

                name_of_file = uri.replace(".", "-").replace(":", "-").replace("/", "-")
                path = newpath + "/"

                completeName = os.path.join(path, name_of_file+".txt")  


                file1 = open(completeName, "w")

                file1.write(text)

                file1.close()
            except:
                pass
